In [1]:
from __future__ import division, print_function, absolute_import, unicode_literals
import numpy as np

import pygsti
# OLD: from pygsti.extras import circuit

/Users/enielse/research/pyGSTi/packages/pygsti/tools/matrixtools.py:23: UserWarning: Could not import Cython extension - falling back to slower pure-python routines
  _warnings.warn("Could not import Cython extension - falling back to slower pure-python routines")


In [2]:
# The most basic initialization of a BasicGateSet object. It is essentially empty
# OLD: bgs = circuit.BasicGateSet(n=2)
bgs = pygsti.construction.build_nqubit_standard_gateset(nQubits=2, gate_names=[])

In [3]:
# A list of standard hard-coded gate labels
# OLD: gllist = ['I', 'H','P','CNOT']
gllist = ['Gi','Gh','Gp','Gcnot','Gt'] 
  # Note: would need to include 'Gt' here (for ordering) if it was to be in this 
  # gate set, but since we're making a *Clifford* gate set below and Gt isn't a 
  # Clifford we can't actually include it.  This would normally result in an
  # error except that below we set on_construction_error='warn'.

# A dictionary of unitaries that do not need to be already known to the code
#OLD: unitaries={'T' : np.array([[1.,0.],[0.,np.exp(np.pi*1j/4)]])}
unitaries={'Gt' : np.array([[1.,0.],[0.,np.exp(np.pi*1j/4)]])}

# The number of qubits
n = 10

# The availiability of gates that are not available to all qubit / qubit pairs
availability = {}

# Let's make a the CNOT gate be connected in a directed ring.
# OLD: a = np.zeros((n,n),int)
# OLD: for i in range(0,n-1):
# OLD:     a[i,i+1] = 1
# OLD: a[n-1,0] = 1
a = [(i,i+1) for i in range(0,n-1)] + [(n-1,0)]

# OLD: availability={'CNOT':a}
availability={'Gcnot':a}

# Specifies whether Clifford-only quantities should be populated for the gates that are Clifford
# OLD: clifford = True
parameterization = 'clifford'

# OLD: BGS = circuit.BasicGateSet(n=n,gllist=gllist,unitaries=unitaries,clifford=clifford)
BGS = pygsti.construction.build_nqubit_standard_gateset(
    nQubits=n, gate_names=gllist, nonstd_gate_unitaries=unitaries,
    availability=availability, parameterization=parameterization,
    on_construction_error='warn')

/Users/enielse/research/pyGSTi/packages/pygsti/construction/gatesetconstruction.py:1225: UserWarning:

Failed to create clifford gate Gt. Dropping it.



In [4]:
# OLD: BGS.smatrix # a dictionary of the smatrices for each "root" gate-name
BGS.get_clifford_symplectic_reps()

{u'Gcnot': (array([[1, 0, 0, 0],
         [1, 1, 0, 0],
         [0, 0, 1, 1],
         [0, 0, 0, 1]]), array([0, 0, 0, 0])), u'Gh': (array([[0, 1],
         [1, 0]]), array([0, 0])), u'Gi': (array([[1, 0],
         [0, 1]]), array([0, 0])), u'Gp': (array([[1, 0],
         [1, 1]]), array([1, 0]))}

In [5]:
# We can also populate a DeviceSpec using the same things. When the device spec is made it populates some
# extra things that are useful for circuit compiling.

# If this is true, if creates compilations for a set of "standard" Clifford gates
# OLD: construct_std_compilations=True
construct_std_compilations = ('paulieq','absolute') # now specify which std compilations you want (this is also the default)

# Todo : the device spec will give an error when the idle is not in the gateset. This is ok, but it should 
# do an assert.

# OLD: ds = circuit.DeviceSpec(n, gllist, unitaries, clifford=True, availability=availability,
# OLD:                         construct_std_compilations=construct_std_compilations, verbosity=0)
ds = pygsti.obj.ProcessorSpec(
    nQubits=n, gate_names=gllist, nonstd_gate_unitaries=unitaries,
    availability=availability, construct_models=("clifford",),
    construct_clifford_compilations=construct_std_compilations, verbosity=0)

In [6]:
# This is a CompilationLibraries object, which holds compilations for, e.g., CNOT between 1 and 4
compilations = ds.compilations

# There are two types of compilation, stored under compilations.paulieq and compilations.absolute.
# The latter is a absolute compilation, i.e., the circuit implements the claimed gate. The former only
# implements the claimed gate up to Paulis.

# OLD: c = ds.compilations.paulieq[circuit.Gate('CNOT',(1,4))]
c = ds.compilations['paulieq'][pygsti.obj.Label('CNOT',(1,4))]

print(c)

# OLD: c = ds.compilations.absolute[circuit.Gate('X',1)]
c = ds.compilations['absolute'][pygsti.obj.Label('X',1)]

print(c)

Qubit 0 ---|  |-|  |-|  |-|  |-|  |-|  |-|  |-|  |---
Qubit 1 ---Gcnot:1:2|-||  |-|  |-|  |-Gcnot:1:2|-||  |-|  |-|  |---
Qubit 2 ---Gcnot:1:2|-|Gcnot:2:3|-||  |-Gcnot:2:3|-|Gcnot:1:2|-|Gcnot:2:3|-||  |-Gcnot:2:3|-|--
Qubit 3 ---|  |-Gcnot:2:3|-|Gcnot:3:4|-|Gcnot:2:3|-||  |-Gcnot:2:3|-|Gcnot:3:4|-|Gcnot:2:3|-|--
Qubit 4 ---|  |-|  |-Gcnot:3:4|-||  |-|  |-|  |-Gcnot:3:4|-||  |---
Qubit 5 ---|  |-|  |-|  |-|  |-|  |-|  |-|  |-|  |---
Qubit 6 ---|  |-|  |-|  |-|  |-|  |-|  |-|  |-|  |---
Qubit 7 ---|  |-|  |-|  |-|  |-|  |-|  |-|  |-|  |---
Qubit 8 ---|  |-|  |-|  |-|  |-|  |-|  |-|  |-|  |---
Qubit 9 ---|  |-|  |-|  |-|  |-|  |-|  |-|  |-|  |---

Qubit 0 ---|  |-|  |-|  |-|  |---
Qubit 1 ---|Gh |-|Gp |-|Gp |-|Gh |---
Qubit 2 ---|  |-|  |-|  |-|  |---
Qubit 3 ---|  |-|  |-|  |-|  |---
Qubit 4 ---|  |-|  |-|  |-|  |---
Qubit 5 ---|  |-|  |-|  |-|  |---
Qubit 6 ---|  |-|  |-|  |-|  |---
Qubit 7 ---|  |-|  |-|  |-|  |---
Qubit 8 ---|  |-|  |-|  |-|  |---
Qubit 9 ---|  |-|  |-|  |-|  |---



In [7]:
# A matrix giving the shorest path between qubits
print(ds.qubitgraph.shortest_path_predecessor_matrix())

# A matrix giving the distance between qubits
print(ds.qubitgraph.shortest_path_distance_matrix())

[[-9999     0     1     2     3     4     7     8     9     0]
 [    1 -9999     1     2     3     4     5     8     9     0]
 [    1     2 -9999     2     3     4     5     6     9     0]
 [    1     2     3 -9999     3     4     5     6     7     0]
 [    1     2     3     4 -9999     4     5     6     7     0]
 [    1     2     3     4     5 -9999     5     6     7     8]
 [    9     2     3     4     5     6 -9999     6     7     8]
 [    9     0     3     4     5     6     7 -9999     7     8]
 [    9     0     1     4     5     6     7     8 -9999     8]
 [    9     0     1     2     3     6     7     8     9 -9999]]
[[0. 1. 2. 3. 4. 5. 4. 3. 2. 1.]
 [1. 0. 1. 2. 3. 4. 5. 4. 3. 2.]
 [2. 1. 0. 1. 2. 3. 4. 5. 4. 3.]
 [3. 2. 1. 0. 1. 2. 3. 4. 5. 4.]
 [4. 3. 2. 1. 0. 1. 2. 3. 4. 5.]
 [5. 4. 3. 2. 1. 0. 1. 2. 3. 4.]
 [4. 5. 4. 3. 2. 1. 0. 1. 2. 3.]
 [3. 4. 5. 4. 3. 2. 1. 0. 1. 2.]
 [2. 3. 4. 5. 4. 3. 2. 1. 0. 1.]
 [1. 2. 3. 4. 5. 4. 3. 2. 1. 0.]]


In [8]:
# OLD: print(ds.compilations.absolute[ds.compilations.absolute.keys()[0]])
print(ds.compilations['absolute'][ds.compilations['absolute'].keys()[0]])
print("Is the compilation for: ",ds.compilations['absolute'].keys()[0])

Qubit 0 ---|Gi |---
Qubit 1 ---|  |---
Qubit 2 ---|  |---
Qubit 3 ---|  |---
Qubit 4 ---|  |---
Qubit 5 ---|  |---
Qubit 6 ---|  |---
Qubit 7 ---|  |---
Qubit 8 ---|  |---
Qubit 9 ---|  |---

Is the compilation for:  I:0


In [9]:
# Device specs can be given "GateSetModel" that is stored in the .models dictionary
# OLD: ds.models["Null model"] = circuit.GateSetModel(operators=None,n=n,mtype='purestate')
ds.models["Null model"] = pygsti.construction.build_nqubit_standard_gateset(
                                nQubits=n, gate_names=[], evotype='statevec', sim_type='map')

In [10]:
# Let's create a new DeviceSpec on less qubits, with full connectivity
# OLD: gllist = ['I', 'H','P','CPHASE']
gllist = ['Gi','Gh','Gp','Gcphase']

# The number of qubits
n = 4

# OLD: ds2 = circuit.DeviceSpec(n, gllist,  clifford=True, verbosity=1)
ds2 = pygsti.obj.ProcessorSpec(n, gllist, construct_models=("clifford",), verbosity=1)

Creating a circuit to implement H up to Pauli gates on qubit 0...
Generating template for a pauli-equivalence compilation of the H gate...
Checking all length 1 1-qubit circuits... (3)
Compilation template created!
Creating a circuit to implement P up to Pauli gates on qubit 0...
Generating template for a pauli-equivalence compilation of the P gate...
Checking all length 1 1-qubit circuits... (3)
Compilation template created!
Creating a circuit to implement PH up to Pauli gates on qubit 0...
Generating template for a pauli-equivalence compilation of the PH gate...
Checking all length 1 1-qubit circuits... (3)
Checking all length 2 1-qubit circuits... (9)
Compilation template created!
Creating a circuit to implement HP up to Pauli gates on qubit 0...
Generating template for a pauli-equivalence compilation of the HP gate...
Checking all length 1 1-qubit circuits... (3)
Checking all length 2 1-qubit circuits... (9)
Compilation template created!
Creating a circuit to implement HPH up to Pa

In [11]:
# We can also create less trivial examples, which then allow us to do circuit simulations (the model
# above fairly obviously won't work if we hand it to a simulator).

# Lets construct a model which has perfect gates. Note that the code below wouldn't be correct
# for a device with non-symmetric gates -- in that case care has to be taken to input the 
# unitaries correctly.
# OLD: operators = {}
# OLD: for gate in ds2.allgates:
# OLD:     operators[gate] = ds2.gateset.unitaries[gate.label]  
# OLD: ds2.models['Target'] = circuit.GateSetModel(operators,n,mtype='purestate')

#In updated pyGSTi, models are just additional gatesets with non-clifford parameterizations
# (which means they can currently be simulated).  "svmap" == "(pure) state vector map-based simulations"
ds2.models['Target'] = pygsti.construction.build_nqubit_standard_gateset(
    n, gllist, parameterization="static", evotype="statevec", sim_type="map")
# OR you could do this (which replaces ds2.models['Target']), since 'Target' is known as a standard model:
ds2.add_std_model('Target')

In [12]:
# OLD: s, p = circuit.random_clifford(4)
# OLD: sin, pin = circuit.inverse_clifford(s,p)
s, p = pygsti.tools.random_clifford(4)
sin, pin = pygsti.tools.inverse_clifford(s,p)

# Creates the circuit, over the gates in ds2.
# OLD: c = circuit.compile_clifford(s, p, ds=ds2)
# OLD: c.append_circuit(circuit.compile_clifford(sin, pin, ds=ds2))

c = pygsti.alg.compile_clifford(s, p, pspec=ds2)
c.append_circuit(pygsti.alg.compile_clifford(sin, pin, pspec=ds2))


#print(c)

In [13]:
# As we can see, the output is (0,0,0,0) with probability 1. (when no input is given to the simulators, the input
# is taken to be (0,0,0,0)).

simout = c.simulate(ds2.models['Target'])
# OLD: print(simout[0,0,0,0])
print(simout['0000'])
x = 0
for key in list(simout.keys()):
    x += simout[key]
# OLD x = x - simout[0,0,0,0]
x = x - simout['0000']
print(x)

0.9999999999999871
0.0


In [14]:
#Tim's example code that wasn't working before - just to make sure...
n = 3
glist = ['Gh','Gp','Gcnot']
pygsti.obj.ProcessorSpec(n,glist)

Creating a circuit to implement H up to Pauli gates on qubit 0...
Generating template for a pauli-equivalence compilation of the H gate...
Checking all length 1 1-qubit circuits... (2)
Compilation template created!
Creating a circuit to implement P up to Pauli gates on qubit 0...
Generating template for a pauli-equivalence compilation of the P gate...
Checking all length 1 1-qubit circuits... (2)
Compilation template created!
Creating a circuit to implement PH up to Pauli gates on qubit 0...
Generating template for a pauli-equivalence compilation of the PH gate...
Checking all length 1 1-qubit circuits... (2)
Checking all length 2 1-qubit circuits... (4)
Compilation template created!
Creating a circuit to implement HP up to Pauli gates on qubit 0...
Generating template for a pauli-equivalence compilation of the HP gate...
Checking all length 1 1-qubit circuits... (2)
Checking all length 2 1-qubit circuits... (4)
Compilation template created!
Creating a circuit to implement HPH up to Pa